# Lab: The Power of Python Decorators

Welcome! Before we dive deeper into advanced LangGraph features, we need to master a core Python concept that makes frameworks like LangGraph so elegant: **Decorators**.

### What is a Decorator?

A decorator is a special kind of function that **takes another function as input** and **returns a modified or extended function as output**. It allows you to add new functionality to an existing function without changing its source code.

Think of it like adding a turbocharger to a car's engine. The engine is still the same, but the turbocharger (the decorator) gives it extra power.

**Why is this important for Agentic AI?**
Decorators are used everywhere in modern AI frameworks to:
- **Register functions as tools** (`@tool`)
- **Add logging or timing** to see how long agent steps take.
- **Implement safety checks** or moderation before or after a function runs.

In [ ]:
print("Running setup...")
# No external libraries needed for this lab, just pure Python!

Running setup...


### Part 1: Functions as Objects

The first thing to understand is that in Python, functions are first-class objects. This means you can pass them around just like you would pass a string or a number.

In [ ]:
def greet(name):
    return f"Hello, {name}!"

print(greet("Alice"))

# You can assign a function to a variable
say_hello = greet
print(say_hello("Bob"))

# You can pass a function as an argument to another function
def process_greeting(greeter_func, person_name):
    print("Processing...")
    print(greeter_func(person_name))
    print("Done!")

process_greeting(greet, "Charlie")

Hello, Alice!
Hello, Bob!
Processing...
Hello, Charlie!
Done!


### Part 2: Building Our First Decorator

Now, let's build a simple decorator from scratch. Our decorator will add a border of `========` characters around the output of any function it decorates.

In [ ]:
# This is the decorator function. It takes a function as input.
def border_decorator(func_to_decorate):
    # It defines a new 'wrapper' function inside.
    # This wrapper is what actually gets called.
    def wrapper(*args, **kwargs):
        print("="*20)
        # It calls the original function that was passed in.
        result = func_to_decorate(*args, **kwargs)
        print(result)
        print("="*20)
        return result # It's good practice to return the original result

    # The decorator returns the new wrapper function.
    return wrapper

# Now, let's create a function we want to decorate
def display_name(name, age):
    return f"Name: {name}, Age: {age}"

print("--- Calling the function normally ---")
print(display_name("David", 30))

print("\n--- Calling the function after decorating it manually ---")
decorated_display_func = border_decorator(display_name)
decorated_display_func("Eve", 25)

--- Calling the function normally ---
Name: David, Age: 30

--- Calling the function after decorating it manually ---
Name: Eve, Age: 25


'Name: Eve, Age: 25'

### Part 3: Using the `@` Syntactic Sugar

The manual process above is a bit clunky. Python provides a much cleaner way to apply a decorator using the `@` symbol. This is called "syntactic sugar" because it makes the code sweeter to read!

In [ ]:
# This does the exact same thing as the manual process before!
@border_decorator
def display_title(title):
    return f"Title: {title}"

@border_decorator
def display_error(error_message):
    return f"ERROR: {error_message}"

print("--- Calling functions decorated with the @ symbol ---")
display_title("Senior Python Developer")
print("\n") # Add a newline for spacing
display_error("File not found")

--- Calling functions decorated with the @ symbol ---
Title: Senior Python Developer


ERROR: File not found


'ERROR: File not found'

### Part 4: Practical Example - A Tool Registry

Now let's see a real-world use case that mirrors how LangChain's `@tool` decorator works. We will create a decorator that automatically registers any function it decorates into a central dictionary of available tools.

In [ ]:
# This dictionary will act as our tool registry
AVAILABLE_TOOLS = {}

def register_tool(func):
    """A decorator that registers a function in our AVAILABLE_TOOLS dict."""
    print(f"---> Registering tool: {func.__name__}")
    AVAILABLE_TOOLS[func.__name__] = func
    return func # Return the original function, as we're not modifying it

@register_tool
def search_linkedin(candidate_name: str) -> str:
    """Simulates searching for a candidate on LinkedIn."""
    return f"Found LinkedIn profile for {candidate_name}."

@register_tool
def schedule_interview(candidate_name: str, date: str) -> str:
    """Simulates scheduling an interview."""
    return f"Interview scheduled for {candidate_name} on {date}."

print("\n--- Available tools after registration ---")
print(AVAILABLE_TOOLS)

print("\n--- Simulating an agent calling a tool ---")
tool_to_call = "schedule_interview"
if tool_to_call in AVAILABLE_TOOLS:
    tool_function = AVAILABLE_TOOLS[tool_to_call]
    result = tool_function("Frank", "2025-10-15")
    print(f"Tool result: {result}")
else:
    print(f"Error: Tool '{tool_to_call}' not found.")

---> Registering tool: search_linkedin
---> Registering tool: schedule_interview

--- Available tools after registration ---
{'search_linkedin': <function search_linkedin at 0x78e0ad884a40>, 'schedule_interview': <function schedule_interview at 0x78e0ad886ac0>}

--- Simulating an agent calling a tool ---
Tool result: Interview scheduled for Frank on 2025-10-15.
